In [2]:
import matplotlib.pyplot as plt
from src.datasets.vindr import VindrLesionDataset
from src.pipeline.breast import BreastImageProcessingPipeline

pipeline = BreastImageProcessingPipeline()
train_dataset = VindrLesionDataset('/media/nvidia/DATA/Data/vindr_mammo/', pipeline, True)
test_dataset = VindrLesionDataset('/media/nvidia/DATA/Data/vindr_mammo/', pipeline, False)